# Fonctions d'ordre supérieur

Python traite les fonctions comme des [citoyens de première-classe](https://en.wikipedia.org/wiki/First-class_citizen).
En particulier, on peut :

* assigner une fonction à une variable
* passer une fonction en argument d'une autre fonction,
* retourner une fonction depuis une fonction.

Voyons quelques exemples utiles de cette caractéristique du langage.

Assigner une fonction à une variable :

In [1]:
def square(x: int) -> int:
    return x ** 2

f = square

f(10)

100

Passer une fonction en argument d'une autre fonction :

In [2]:
from collections.abc import Callable

def greet(name: str, f: Callable[[str], str]) -> str:
    return f"Hello {f(name)}!"

greet("John", str.upper)

'Hello JOHN!'

Retourner une fonction depuis une fonction :

In [3]:
from collections.abc import Callable

def partial_add(a: int) -> Callable[[int], int]:
    def add(b: int) -> int:
        return a + b
    return add

add10 = partial_add(10)
add10(5)

15

L'exemple ci-dessus réalise une [curryfication](https://fr.wikipedia.org/wiki/Curryfication), du nom du mathématicien [Haskell Curry](https://fr.wikipedia.org/wiki/Haskell_Curry).

On aurait pu utiliser les fonctions `partial` et `add` pour accomplir le même objectif :

In [4]:
from functools import partial
from operator import add

add10 = partial(add, 10)
add10(5)

15

## Les fonctions anonymes

On les appelle souvent [lambdas](https://fr.wikipedia.org/wiki/Fonction_anonyme).
Il s'agit de fonctions qui n'ont pas de nom.
En général, elles sont définies directement dans le contexte où elles sont utilisées.
Comme les lambdas sont des fonctions, on peut les utiliser partout où une fonction est attendue, essentiellement pour gagner en concision.

Comme les fonctions classiques, on peut assigner une lambda à une variable.

In [5]:
to_square = lambda x: x ** 2
to_square(2)

4

Les lambdas sont souvent passées en argument d'une autre fonction.
Par exemple, le premier argument de la fonction `map` doit être une fonction qui à chaque élément d'un [iterable](https://docs.python.org/3.15/glossary.html#term-iterable) fait correspondre un nouvel élément.

In [6]:
nums = [1, 2, 3, 4, 5, 6]
squares = list(map(lambda x: x ** 2, nums))
squares

[1, 4, 9, 16, 25, 36]

In [7]:
words = ["Hello", "World"]
upper = list(map(lambda word: word.upper(), words))
upper

['HELLO', 'WORLD']

Ce dernier exemple peut même se passer de lambda.
Il est encore plus simple de passer directement la fonction `str.upper` en argument.

In [8]:
words = ["Hello", "World"]
upper = list(map(str.upper, words))
upper

['HELLO', 'WORLD']

Comme pour `map`, la fonction `filter` admet comme premier argument une fonction qui pour chaque élément d'une iterable retourne un `bool`.
Si la fonction retourne `true`, alors l'élément est conservé dans l'itérateur.

In [9]:
nums = [1, 2, 3, 4, 5, 6]
squares_of_even = list(map(lambda x: x ** 2, filter(lambda x: x % 2 == 0, nums)))
squares_of_even

[4, 16, 36]

Dans l'exemple ci-dessus, qui combine `filter` et `map`, on pourra argumenter sur la lourdeur de la syntaxe, notamment comparé à une [compréhension de liste](https://fr.wikipedia.org/wiki/Liste_en_compr%C3%A9hension) :

In [10]:
nums = [1, 2, 3, 4, 5, 6]
squares_of_even = [x ** 2 for x in nums if x % 2 == 0]
squares_of_even

[4, 16, 36]

La fonction [`reduce`](https://docs.python.org/3/library/functools.html) permet de combiner ensemble les éléments d'un iterable.
Le premier argument de la fonction `reduce` est une fonction qui indique pour un couple d'éléments comment il convient de les combiner.

In [11]:
from functools import reduce

a = [1, 2, 3, 4, 5, 6]
sum_of_squares = reduce(lambda x, y: x + y, map(lambda x: x ** 2, a), 0)
sum_of_squares

91

On peut améliorer la lisibilité de l'exemple précédent, en important la fonction `add`, qui remplace avantagement la première lambda :

In [12]:
from functools import reduce
from operator import add

a = [1, 2, 3, 4, 5, 6]
sum_of_squares = reduce(add, map(lambda x: x ** 2, a), 0)
sum_of_squares

91

La fonction `sorted` permet de trier un iterable, au moyen d'une fonction qui retourne la clé d'indexation de l'élément dans la liste retournée.

In [13]:
class Person:

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    def __repr__(self):
        return f"{self.first_name} {self.last_name}"

pythons = [Person("Graham", "Chapman"),
           Person("John", "Cleese"),
           Person("Eric", "Idle"),
           Person("Michael", "Palin"),
           Person("Terry", "Jones"),
           Person("Terry", "Gilliam")]

sorted_pythons = sorted(pythons, key=lambda p: (p.first_name, p.last_name))
sorted_pythons

[Eric Idle,
 Graham Chapman,
 John Cleese,
 Michael Palin,
 Terry Gilliam,
 Terry Jones]

Pour aller plus loin :

* https://docs.python.org/3.15/library/functools.html
* https://docs.python.org/3.15/howto/sorting.html

## Développer des fonctions d'ordre supérieur

Une [fonction d'ordre supérieur](https://fr.wikipedia.org/wiki/Fonction_d%27ordre_sup%C3%A9rieur) est une fonction qui admet au moins l'une de ces 2 caractéristiques :

* Elle prend une autre fonction en argument
* Elle retourne une fonction.

La bibliothèque standard de Python contient déjà des fonctions d'ordre supérieur comme `map`, `filter` et `reduce`, mais nous pouvons aussi développer les nôtres, pour implémenter des solutions élégantes et compactes.

Dans l'exemple ci-dessous, `execute` exécute la fonction `sum_of_ints` passée en argument et détermine le temps de réponse.

In [14]:
import time

def execute(f):
    start = time.monotonic()
    result = f()
    end = time.monotonic()
    return (result, end - start)

def sum_of_ints() -> int:
    """Une fonction quelconque qui retourne la somme des entiers jusqu'à 1 000 000."""
    result = 0
    for i in range(1000000):
        result += i
    return result

(result, duration) = execute(sum_of_ints)

print(f"L'exécution de la fonction {sum_of_ints.__name__} a duré {duration:.3f} s et a retourné {result}.")

L'exécution de la fonction sum_of_ints a duré 0.062 s et a retourné 499999500000.


Dans les exemples ci-dessous, `greeter` et `die` retournent une [closure](https://fr.wikipedia.org/wiki/Fermeture_(informatique)).
Par exemple une lambda retournée par `greeter` encapsule l'argument `greeting` passé en argument de `greeter`.

In [15]:
from collections.abc import Callable

def greeter(greeting: str) -> Callable[[str], str]:
    return lambda name: f"{greeting}, {name}!"

hello = greeter("Hello")
hello("John")

'Hello, John!'

In [16]:
from collections.abc import Callable
from random import randint
from collections import Counter

def die(sides: int) -> Callable[[], int]:
    return lambda: randint(1, sides)

d6 = die(6)
# Lancer 100 fois le d6 et compter le nombre de 1, de 2, etc.
print(Counter([d6() for _ in range(100)]))

Counter({4: 20, 2: 19, 6: 18, 5: 16, 3: 14, 1: 13})
